In [ ]:
%pip install pandas # pour manipuler des tableaux de données (dataframes)
%pip install openpyxl # pour lire les fichiers Excel .xlsx
%pip install requests # pour envoyer des requêtes HTTP (aller chercher une page web)
%pip install beautifulsoup4 # pour lire une page web
%pip install scikit-learn
%pip install nltk # pour le traitement du langage naturel
%pip install matplotlib seaborn
%pip install wordcloud
%pip install networkx

Navigation pour la collecte des données

In [19]:
from pathlib import Path # pour manipuler les chemins de fichiers
import pandas as pd # pour manipuler des tableaux de données (dataframes)
import requests # Importe la librairie pour envoyer des requêtes HTTP (aller chercher une page web)

# En-têtes HTTP : Se faire passer pour un vrai navigateur pour éviter certains blocages anti-bot
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

# Chemin vers le fichier Excel
excel_path = Path("Organisations.xlsx")

# Vérifie si le fichier excel existe sinon le script s'arrête
if excel_path.exists():
    print("Excel trouvé :", excel_path.resolve())
else:
    print("Excel non trouvé :", excel_path.resolve())
    raise FileNotFoundError()

# Lire le fichier Excel et le mettre dans une variable df_excel sous forme de tableau pandas (un DataFrame).
df_excel = pd.read_excel(excel_path)
print()
print("Aperçu des données Excel :")
print(df_excel.head(5)) # Affiche les 5 premières lignes du tableau
print()

# Détection intelligente des colonnes

# Nettoyer les noms de colonnes forcant en string et enlevant les espaces
df_excel.columns = df_excel.columns.astype(str).str.strip()
print(df_excel.columns.tolist())

# Fonction pour trouver la colonne correspondant parmi des noms possibles sinon retourner la colonne par défaut (peut etre suprimée)
def trouver_colonne(noms_possibles, colonne_defaut):
    for nom_colonne in df_excel.columns:
        if nom_colonne.lower() in noms_possibles:
            return nom_colonne
    return df_excel.columns[colonne_defaut]

colone_noms = trouver_colonne({"name", "nom", "organisation", "organization"}, 0)
colone_url  = trouver_colonne({"url", "link", "website", "site", "web", "homepage"}, 1)
print()
print("Colonnes détectées :" + "\n Nom: " + colone_noms + "\n url: " + colone_url)

# Fonction pour nettoyer les URLs (peut être suprimée)
def nettoyer_url(url: str) -> str:
    # Enlève les espaces au début/fin
    if pd.isna(url):
        return ""
    url = url.strip()
    # Si vide ou "nan", on ignore
    if url == "" or url.lower() == "nan":
        return ""
    # Si l'URL n'a pas http:// ou https://, on l'ajoute
    if not url.lower().startswith(("http://", "https://")):
        url = "https://" + url
    return url

# Extraire les colonnes spécifiques en listes (peut être suprimée)
Noms = df_excel[colone_noms].tolist()
urls = df_excel[colone_url].tolist()

def read_html_url(url: str):
    # Essayer d'envoyer une requête GET au site web (télécharge le contenu de la page)
    try:
        response = requests.get(url, headers=headers, timeout=5, allow_redirects=True)
        if response.status_code != 200:
            return response.status_code, None
        # Optionnel c'est si Requests ne sait pas quel encodage utiliser, alors elle utilise l’encodage deviné automatiquement (peut être suprimée)
        if not response.encoding:
            response.encoding = response.apparent_encoding
        return response.status_code, response.text
    except requests.Timeout:
        return "Erreur Timeout", None
    except requests.ConnectionError:
        return "Erreur Connexion", None 
    except requests.HTTPError:
        return "Erreur HTTP", None
    except requests.TooManyRedirects:
        return "Trop de redirections", None
    except requests.URLRequired:
        return "URL invalide", None
    except requests.RequestException:
        return "Autre erreur", None

for nom, url in zip(Noms, urls):
    url = nettoyer_url(url)
    if not url:
        continue
    status_code, texte = read_html_url(url)
    print("\nNom :", nom)
    print("URL :", url)
    print("Statut de la requête :", status_code)
    # Affiche les 100 premiers caractères du contenu HTML
    if texte is None:
            print("Pas de contenu car erreur lors de la requête.")
            continue
    if len(texte) < 100:
        print("contenue faible :", texte)
    else:
        print("Contenu de la page (extrait) :", texte[:100])

Excel trouvé : C:\Users\krack\Desktop\Data collection\Organisations.xlsx

Aperçu des données Excel :
        Name                                    URL
0      Adobe      https://www.adobe.com/about-adobe
1     Google                   https://about.google
2  Microsoft  https://www.microsoft.com/en-us/about
3     Amazon   https://www.aboutamazon.com/about-us
4      Apple       https://www.apple.com/leadership

['Name', 'URL']

Colonnes détectées :
 Nom: Name
 url: URL

Nom : Adobe
URL : https://www.adobe.com/about-adobe
Statut de la requête : Erreur Timeout
Pas de contenu car erreur lors de la requête.

Nom : Google
URL : https://about.google
Statut de la requête : 200
Contenu de la page (extrait) : <!doctype html>
<html class="page" dir="ltr" lang="en-US" locale="en-US">
  <head>
    <meta charset

Nom : Microsoft
URL : https://www.microsoft.com/en-us/about
Statut de la requête : 200
Contenu de la page (extrait) : <!DOCTYPE html>

<html lang="en-US" dir="ltr">
  <head>
    
    

    

KeyboardInterrupt: 

Extraction du contenu HTML

In [ ]:
import bs4    # Outil pour lire une page web
def make_soup(content: str):
    texte = bs4.BeautifulSoup(content, "html.parser") # Transforme le HTML en structure lisible par Python
    return texte

In [ ]:
import re # gére les expressions régulières qui permettent de manipuler du texte (regex)
def extract_texte(texte):
    for tag in texte(["script", "style", "noscript"]):
        tag.decompose()

    # priorité au contenu dans <main> si présent (souvent beaucoup plus propre)
    main = texte.find("main")
    if main:
        text = main.get_text(separator=" ")
    else:
        text = texte.get_text(separator=" ")

    text = re.sub(r"\s+", " ", text).strip()
    return text 

Stockage des données (CSV et JSON)

In [ ]:
results = []
corpus = {}
Doublons = set()
for nom, url in zip(Noms, urls):
    url = nettoyer_url(url)
    if not url:
        continue

    nom = str(nom).strip()
    if not nom:
        continue

    # filtre doublons (on garde le 1er)
    if nom in Doublons:
        continue
    Doublons.add(nom)

    status_code, texte = read_html_url(url)

    print("\nNom :", nom)
    print("URL :", url)
    print("Statut de la requête :", status_code)
    # Affiche les 100 premiers caractères du contenu HTML
    if texte is None:
            print("Pas de contenu car erreur lors de la requête.")
            continue
    texte = make_soup(texte)
    texte = extract_texte(texte)
    if len(texte) < 50:
        print("contenue tres faible et insufisant :", texte)
        continue
    if len(texte) < 100:
        print("contenue faible :", texte)
    else:
        print("Contenu de la page (extrait) :", texte[:100])

    # dictionnaire[cle] = valeur
    corpus[nom] = texte
    texte_originale = corpus[nom]

    results.append({
        "name": nom,
        "url": url,
        "http_status": status_code,
        "text": texte,
        "text_length": len(texte)
    })

# 5 Export
df_excel_out = pd.DataFrame(results)
df_excel_out.to_csv("about_texts.csv", index=False, encoding="utf-8")
df_excel_out.to_json("about_texts.json", orient="records", force_ascii=False, indent=2)
print("\nDonnées exportées vers about_texts.csv et about_texts.json")


Nom : Adobe
URL : https://www.adobe.com/about-adobe
Statut de la requête : Erreur Timeout
Pas de contenu car erreur lors de la requête.

Nom : Google
URL : https://about.google
Statut de la requête : 200
Contenu de la page (extrait) : Gemini 3 Flash: Bring any idea to life faster Our latest model with frontier intelligence built for 

Nom : Microsoft
URL : https://www.microsoft.com/en-us/about
Statut de la requête : 200
Contenu de la page (extrait) : We empower the world Microsoft’s mission is to empower every person and every organization on the pl

Nom : Amazon
URL : https://www.aboutamazon.com/about-us
Statut de la requête : 200
Contenu de la page (extrait) : About Us | About Amazon News Stores and Shopping AWS Devices Entertainment Artificial Intelligence W

Nom : Apple
URL : https://www.apple.com/leadership
Statut de la requête : 200
Contenu de la page (extrait) : Executive Profiles Tim Cook CEO Katherine Adams Senior Vice President and General Counsel Eddy Cue S

Nom : Spotify
U

Informations : Nombre d'URLs, nombre de documents et corpus

In [ ]:
df_clean = df_excel_out
print(len(urls))
print(len(df_clean))
print(len(corpus))

df_clean = df_clean.reset_index(drop=True)

print("Nb documents:", len(df_clean))
print(df_clean.head(8)[["name", "url", "text_length"]])
print('Corpus entier '+ str(corpus))

Nettoyer et Tokeniser

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

stop_words = list(set(stopwords.words('english'))) + ["'s"]
stem = nltk.stem.SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()

def nettoyer_texte(texte: str) -> str:
    texte = texte.lower()
    texte = re.sub(r"https?://\S+", " ", texte) # enlever les URLs
    texte = re.sub(r"www\.\S+", " ", texte) # enlever les URLs
    texte = re.sub(r"\d+", " ", texte) # enlever les chiffres
    texte = re.sub(r"[^a-z\s]", " ", texte) # remplacer tout ce qui n’est pas lettre ou espace par un espace
    texte = re.sub(r"\s+", " ", texte).strip() # réduire les espaces
    return texte

def tokenizer_stem(texte: str):
    tokens = word_tokenize(texte)
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    tokens = [token for token in tokens if token.isalpha()]
    tokens = [stem.stem(token) for token in tokens]
    return tokens

def tokenizer_lemma(texte: str):
    tokens = word_tokenize(texte)
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    tokens = [token for token in tokens if token.isalpha()]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

C'est deux autres manières de traiter une liste de documents (nettoyer → tokeniser → stocker/afficher)

In [ ]:
df_nettoyer = df_clean["text"].apply(nettoyer_texte)
df_tokenizer_stem = df_nettoyer.apply(tokenizer_stem)
df_tokenizer_lemma = df_nettoyer.apply(tokenizer_lemma)
print()

for i in range(5):
    print(df_clean["name"].iloc[i])
    print("Texte original :", df_clean["text"].iloc[i])
    print("Texte nettoyé  :", df_nettoyer.iloc[i])
    print("Tokens stemmés :", df_tokenizer_stem.iloc[i])
    print("Tokens lemmatisés :", df_tokenizer_lemma.iloc[i])
    print()

In [ ]:
corpus_nettoyer = {}
corpus_tokenizer_stem = {}
corpus_tokenizer_lemma = {}

# Nettoyage : corpus (brut) vers corpus_nettoyer
for Name in corpus:
    texte = corpus[Name]
    corpus_nettoyer[Name] = nettoyer_texte(texte)
    texte_nettoyer  = corpus_nettoyer[Name]
    
# Tokenisation : corpus_nettoyer -> stem + lemma
for Name in corpus_nettoyer:
    texte = corpus_nettoyer[Name]
    corpus_tokenizer_stem[Name] = tokenizer_stem(texte)
    corpus_tokenizer_lemma[Name] = tokenizer_lemma(texte)
    tokens_stem  = corpus_tokenizer_stem[Name]
    tokens_lemma = corpus_tokenizer_lemma[Name]
    print(Name)
    print("Texte original :", corpus[Name])
    print("Texte nettoyé  :", corpus_nettoyer[Name])
    print("Tokens stemmés :", tokens_stem)
    print("Tokens lemmatisés :", tokens_lemma)
    print()

Matrice terme-document 

In [ ]:
# Counter permet de compter les occurrences des éléments dans une liste
from collections import Counter
# Pour manipuler des tableaux de données
import pandas as pd

# step 1 : Prends chaque liste de tokens de chaque document, puis prends chaque token dans cette liste, 
# et mets tous ces tokens dans un set = pas de doublons.
vocabulary = set()
for tokens_stem in corpus_tokenizer_stem.values():
    for token in tokens_stem:
        vocabulary.add(token)
print(vocabulary)
print()

# Step 2: compte le nombre d'occurrences de chaque token dans chaque document/page
token_frequencies = {}
for Name, tokens_stem in corpus_tokenizer_stem.items():
    token_frequencies[Name] = Counter(tokens_stem)
print(token_frequencies)
print()

data = {}
for token in vocabulary:
    data[token] = []
    for document in corpus_tokenizer_stem:
        nombre = token_frequencies[document].get(token, 0)
        data[token].append(nombre)
print(data)
print()

td_matrix = pd.DataFrame(data, index=corpus_tokenizer_stem.keys())
display(td_matrix.head())
nb_doc = len(td_matrix)
print("Nombre de documents :", nb_doc)

Matrice terme-document filtrée

In [ ]:
# arrondie
seuil_bas = int(nb_doc*0.1)+1
seuil_haut = int(nb_doc*0.9)
print("Nombre de documents :", nb_doc)
print("Seuil bas 10% des documents :", seuil_bas)
print("Seuil haut 90% des documents :", seuil_haut)

# Step 4: Enlever les mots qui apparaissent dans moins de 2 documents ()
present = td_matrix > 0
page_frequency = present.sum(axis=0)
filtrage_td_matrix = td_matrix.loc[:, page_frequency >= seuil_bas]

# Step 4: enlever les mots qui apparaissent dans tous les documents
present = filtrage_td_matrix > 0
page_frequency = present.sum(axis=0)
filtrage_td_matrix = filtrage_td_matrix.loc[:, page_frequency < nb_doc]

filtrage_vocabulary = filtrage_td_matrix.columns.tolist()

corpus_tokenizer_stem_filtre = {}
for nom_doc, tokens_stem in corpus_tokenizer_stem.items():
    corpus_tokenizer_stem_filtre[nom_doc] = [token for token in tokens_stem if token in filtrage_vocabulary]

token_filtrage_frequencies = {}
for Name, tokens_filtrees in corpus_tokenizer_stem_filtre.items():
    token_filtrage_frequencies[Name] = Counter(tokens_filtrees)
print(token_filtrage_frequencies)
print()

print(filtrage_td_matrix.head())
print(filtrage_vocabulary)
print()

# Print le filtrage de la term-document matrix
display(filtrage_td_matrix.head())

Matrice TF-IDF

In [ ]:
import numpy as np

row_sums = filtrage_td_matrix.sum(axis=1)  # Total tokens par page/document
tf = filtrage_td_matrix.div(row_sums, axis=0)
df = (filtrage_td_matrix > 0).sum(axis=0)  
N = filtrage_td_matrix.shape[0]  
idf = np.log((N) / (df))
tf_idf = tf.mul(idf, axis=1)
display(tf_idf.head())

Matrice de similarité cosinus

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

# matrice de similarité cosinus
similarity_matrix_tfidf = cosine_similarity(tf_idf)

# convertir en DataFrame pour une meilleure lisibilité (peut etre suprimée)
similarity_df_tfidf = pd.DataFrame(similarity_matrix_tfidf, index=tf_idf.index, columns=tf_idf.index)

# affiche le DataFrame de similarité
#display(similarity_df_tfidf)

def plot_similarity_matrix(similarity_df_tfidf):
    # Affiche la matrice de similarité
    plt.figure(figsize=(25, 25))
    plt.imshow(similarity_df_tfidf, aspect="auto")
    plt.colorbar(label='Cosine Similarity')
    plt.title('Page internet (Similarity Matrix)')
    plt.xticks(ticks=range(len(similarity_df_tfidf.columns)), labels=similarity_df_tfidf.columns, rotation=90)
    plt.yticks(ticks=range(len(similarity_df_tfidf.index)), labels=similarity_df_tfidf.index)
    plt.xlabel('Page internet')
    plt.ylabel('Page internet')
    plt.show()

    # optionnellement, annoter les cellules avec les valeurs de similarité
    for i in range(len(similarity_df_tfidf)):
        for j in range(len(similarity_df_tfidf)):
            plt.text(j, i, f"{similarity_df_tfidf.iloc[i, j]:.3f}", ha='center', va='center', color='white')
    plt.tight_layout()

display(similarity_df_tfidf.head())

plot_similarity_matrix(similarity_df_tfidf)

names = similarity_df_tfidf.index.tolist()
print(names)

Classement de la similarité cosinus

In [ ]:
# top paires hors diagonale
liste_similarites = []
nb_doc = similarity_matrix_tfidf.shape[0]
for i in range(nb_doc):
    for j in range(i+1, nb_doc):
        score = similarity_matrix_tfidf[i, j]
        doc_a = names[i]
        doc_b = names[j]
        liste_similarites.append((score, doc_a, doc_b))

liste_similarites.sort(reverse=True, key=lambda x: x[0])

print("Top 10 similarités (cosinus) :")
for score, doc_a, doc_b in liste_similarites[:10]:
    print(f"{score:.3f}  {doc_a} <-> {doc_b}")

Analyse descriptive

In [ ]:
from wordcloud import WordCloud
from itertools import combinations

# TOP 20 TF-IDF MOYEN
tfidf_moyen = tf_idf.mean(axis=0).sort_values(ascending=False)

print("Top 20 termes (TF-IDF moyen) :")
for mot, score in tfidf_moyen.head(20).items():
    print(mot, round(score, 4))

# FREQUENCE GLOBALE (sur la matrice filtrée)
frequence_globale = {}  # dictionnaire : mot -> total

for mot in filtrage_td_matrix.columns:                 # chaque mot (colonne)
    total = filtrage_td_matrix[mot].sum()              # somme sur tous les docs
    frequence_globale[mot] = int(total)                # on garde un entier

# Afficher les 20 mots les plus fréquents
freq_triee = sorted(frequence_globale.items(), key=lambda x: x[1], reverse=True)

print("\nTop 20 mots les plus fréquents :")
for mot, nb in freq_triee[:20]:
    print(mot, nb)

# WORDCLOUD (taille = fréquence)
nuage = WordCloud(width=1200, height=600, background_color="white")
nuage = nuage.generate_from_frequencies(frequence_globale)

plt.figure(figsize=(12, 6))
plt.imshow(nuage, interpolation="bilinear")
plt.axis("off")
plt.title("WordCloud (fréquence globale)")
plt.show()

# COOCCURRENCES (par document) par exemple "mot A et mot B apparaissent ensemble dans combien de documents ?"
cooccurrences = Counter()  # (motA, motB) -> nb_documents

for nom_doc, tokens in corpus_tokenizer_stem_filtre.items():
    mots_uniques = sorted(set(tokens))  # une seule fois par doc (pas de biais si répété)
    
    for paire in combinations(mots_uniques, 2):  # toutes les paires possibles
        cooccurrences[paire] += 1

print("\nTop 20 cooccurrences (document-level) :")
for (motA, motB), nb_docs in cooccurrences.most_common(20):
    print(motA, "-", motB, ":", nb_docs)

Analyse sémantique

In [ ]:
from sentence_transformers import SentenceTransformer

# Noms + textes (texte brut)
doc_names = list(corpus_nettoyer.keys())
documents = list(corpus_nettoyer.values())

# Modèle SBERT
model = SentenceTransformer("all-MiniLM-L6-v2")

# Embeddings (vecteurs sémantiques)
embeddings = model.encode(documents, normalize_embeddings=True)

# Similarité cosinus
similarity_matrix_bert = cosine_similarity(embeddings)

# DataFrame lisible
similarity_df_bert = pd.DataFrame(
    similarity_matrix_bert,
    index=doc_names,
    columns=doc_names
)

display(similarity_df_bert.head())
plot_similarity_matrix(similarity_df_bert)

Clustering

In [ ]:
from sklearn.cluster import KMeans

# Nombres de clusters
k = 2

# Clustering sur TF-IDF
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
labels = kmeans.fit_predict(tf_idf)

# Mettre le résultat dans un tableau lisible
df_clusters = pd.DataFrame({
    "document": tf_idf.index,
    "cluster": labels
})

# 4) Voir les documents par groupe
print(df_clusters.sort_values("cluster"))

In [ ]:
from sklearn.decomposition import PCA

# Réduire tf_idf en 2 dimensions
pca = PCA(n_components=2, random_state=42)
points_2d = pca.fit_transform(tf_idf.values)

# plot
plt.figure(figsize=(10, 7))
plt.scatter(points_2d[:, 0], points_2d[:, 1], c=labels)
plt.title("Clustering des documents (TF-IDF + PCA)")
plt.xlabel("Axe 1 (PCA)")
plt.ylabel("Axe 2 (PCA)")
plt.show()

for c in sorted(df_clusters["cluster"].unique()):
    print()
    print("CLUSTER", c)
    print()
    docs = df_clusters[df_clusters["cluster"] == c]["document"].tolist()
    for d in docs:
        print("-", d)

Link Analysis

In [ ]:
import networkx as nx
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urldefrag

# Noeuds correspond à toutes les URLs qu'on a gardées après la data collection, on défintit les noeuds du graphe
urls = df_excel_out["url"].dropna().astype(str).str.strip().tolist()
urls = [u for u in urls if u != ""]
urls_set = set(urls)  # enlève doublons

print("Nb noeuds (URLs) :", len(urls_set))

# Trouver les liens sortants d'une page et garder seulement ceux dans notre liste)
def liens_vers_notre_liste(html, base_url):
    soup = BeautifulSoup(html, "html.parser")
    liens = set()

    for a in soup.find_all("a", href=True):
        href = a["href"].strip()

        # on filtre les liens qui ne sont pas des liens web : ancre interne à la meme page + mail + tel + javascript
        if href.startswith("#") or href.startswith("mailto:") or href.startswith("tel:") or href.startswith("javascript:"):
            continue

        lien = urljoin(base_url, href)      # reconstruit le lien absolu
        lien = urldefrag(lien)[0]           # enlève le #...
        lien = lien.strip()

        # on garde uniquement si c'est une URL de notre liste
        if lien in urls_set:
            liens.add(lien)

    return list(liens)

# Construire les arêtes (liens entre pages)
edges = []

for url in urls_set:
    status_code, html = read_html_url(url) # lire le contenu de la page
    if status_code != 200 or html == "" or html is None: # pas de contenu, on ignore
        continue

    targets = liens_vers_notre_liste(html, url) # trouver les liens vers notre liste
    for t in targets:
        edges.append((url, t))

print("Nb liens internes trouvés :", len(edges))

# Création du graphe orienté
G = nx.DiGraph()
G.add_nodes_from(urls_set)   # même sans liens
G.add_edges_from(edges)

print("Noeuds :", G.number_of_nodes(), "| liens :", G.number_of_edges())

In [ ]:
import pandas as pd
import networkx as nx

# Degree centrality (ici: in/out degree)
in_deg = dict(G.in_degree())
out_deg = dict(G.out_degree())

# Betweenness centrality
bet = nx.betweenness_centrality(G, normalized=True)

# PageRank
pr = nx.pagerank(G)

# Shortest path (sur la plus grande composante)
# Sinon, si le graphe pas connecté => distances infinies
components = list(nx.weakly_connected_components(G)) # toutes les composantes connexes
biggest = max(components, key=len) if len(components) > 0 else set() # composante la plus grande
G_big = G.subgraph(biggest).to_undirected() # graphe non orienté de la plus grande composante

if G_big.number_of_nodes() >= 2: # au moins 2 noeuds connectés
    avg_sp = nx.average_shortest_path_length(G_big) # distance moyenne
    diam = nx.diameter(G_big) # diamètre du graphe
    print("Shortest path (plus grande composante) :")
    print("- distance moyenne :", round(avg_sp, 2))
    print("- diamètre :", diam)
else:
    print("Shortest path : impossible (pas assez de noeuds connectés).")

# Résultat dans un tableau
df_link = pd.DataFrame({
    "url": list(G.nodes()),
    "in_degree": [in_deg[u] for u in G.nodes()],
    "out_degree": [out_deg[u] for u in G.nodes()],
    "betweenness": [bet[u] for u in G.nodes()],
    "pagerank": [pr[u] for u in G.nodes()],
})

df_link["degree_total"] = df_link["in_degree"] + df_link["out_degree"]

print("\nTop 10 PageRank :")
display(df_link.sort_values("pagerank", ascending=False).head(10))

print("\nTop 10 Betweenness :")
display(df_link.sort_values("betweenness", ascending=False).head(10))

print("\nTop 10 In-degree :")
display(df_link.sort_values("in_degree", ascending=False).head(10))

# Export du graphe Gephi
nx.set_node_attributes(G, in_deg, "in_degree")
nx.set_node_attributes(G, out_deg, "out_degree")
nx.set_node_attributes(G, bet, "betweenness")
nx.set_node_attributes(G, pr, "pagerank")

nx.write_gexf(G, "web_graph.gexf")